本文来探索一下python中提供的各种数据保存格式的性能如何。主要以一个 ndarray 格式的数据进行处理分析。包括下面几种方式：

- .bin格式, np.tofile() 和 np.fromfile()
- .npy格式，np.save() 和 np.load()
- .txt 或者 .csv格式，np.savetxt() 和 np.loadtxt()
- .h5 格式，h5py.File(，'r' 或者 'w') 
- .pkl 格式， pickle.dump()和pickle.load()

In [1]:
import numpy as np
from __future__ import print_function
import time

In [20]:
a = np.random.randint(0, 100, size=(10000, 5000))
print(a.dtype, a.shape)
print(a[:2])

int64 (10000, 5000)
[[90 96 38 ..., 67 40 79]
 [40 12 71 ..., 64 76 15]]


### 1.tofile() 和 fromfile()

In [21]:
%time a.tofile('data/a.bin')
%time b = np.fromfile('data/a.bin', dtype=np.int64)
print(b.shape)
print(b[:2])

CPU times: user 4 ms, sys: 392 ms, total: 396 ms
Wall time: 2.06 s
CPU times: user 4 ms, sys: 156 ms, total: 160 ms
Wall time: 160 ms
(50000000,)
[90 96]


- 读入数据的时候要正确设置 dtype 参数

- 读入的数据是一维的，还需要经过 reshape 处理。

### 2.save()和load()

In [22]:
%time np.save('data/a.npy', a)
%time b = np.load('data/a.npy')
print(b.shape)
print(b[:2])

CPU times: user 0 ns, sys: 272 ms, total: 272 ms
Wall time: 269 ms
CPU times: user 0 ns, sys: 116 ms, total: 116 ms
Wall time: 116 ms
(10000, 5000)
[[90 96 38 ..., 67 40 79]
 [40 12 71 ..., 64 76 15]]


- NumPy专用的二进制格式保存数据，它们会自动处理元素类型和形状等信息

### 3.savetxt()和loadtxt()

In [24]:
%time np.savetxt('data/a.txt', a, fmt='%d', delimiter=',')  # 设置以整数形式存储，以逗号隔开
%time b = np.loadtxt('data/a.txt', delimiter=',')
print(b.shape)
print(b[:2])

CPU times: user 18.4 s, sys: 60 ms, total: 18.5 s
Wall time: 18.4 s
CPU times: user 55.9 s, sys: 476 ms, total: 56.4 s
Wall time: 56.3 s
(10000, 5000)
[[ 90.  96.  38. ...,  67.  40.  79.]
 [ 40.  12.  71. ...,  64.  76.  15.]]


- 读写1维和2维数组的文本文件

### 4.h5py.File()

In [25]:
import h5py

time0 = time.time()
f = h5py.File('data/a.h5','w')   #创建一个h5文件，文件指针是f  
f['data'] = a                    #将数据写入文件的主键data下面  
f.close()                        #关闭文件  
print('saving time %.2fs' % (time.time() - time0))

time cost 0.26s


In [26]:
time0 = time.time()
f = h5py.File('data/a.h5','r')   #打开h5文件  
# print f.keys()                      #可以查看所有的主键  
b = f['data'][:]                    #取出主键为data的所有的键值  
f.close()  
print('loading time %.2fs' % (time.time() - time0))
print(b.shape)
print(b[:2])

loading time 0.15s
(10000, 5000)
[[90 96 38 ..., 67 40 79]
 [40 12 71 ..., 64 76 15]]


### 5.pickle.dump()和pickle.load()

In [27]:
import pickle
with open('data/a.pkl', 'wb') as outp:
    %time pickle.dump(a, outp)
    
with open('data/a.pkl', 'rb') as inp:
    %time b = pickle.load(inp)
    
print(b.shape)
print(b[:2])

CPU times: user 30.1 s, sys: 5.92 s, total: 36.1 s
Wall time: 36.1 s
CPU times: user 6.84 s, sys: 10.4 s, total: 17.2 s
Wall time: 17.2 s
(10000, 5000)
[[90 96 38 ..., 67 40 79]
 [40 12 71 ..., 64 76 15]]


## 结果统计

|方法|文件格式|大小|写入耗时|读入耗时|需要处理类型?|需要处理形状?|
|----------|:------|:----------:|---------:|----------:|:---------:|:---------:|
|tofile()|.bin|381.47MB|2.06 s|160 ms|是|是|
|**save()**|.npy|**381.47MB**|269 ms|**116 ms**|否|否|
|savetxt()|.txt|**138.28MB**|18.4 s|56.3 s|是|否|
|**h5py.File()**|.h5|**381.47MB**|**260ms**|150ms|否|否|
|pickle.dump()|.pkl|1.39GB|36.1 s|17.2 s|否|否|

- 总体上最优的方法是使用 np.save() 和 h5py.File(, 'r' or 'w');np.savez() 可以存储多个值， 而 h5py 提供字典的方式存储更是非常方便。
- txt 和 csv 文本文件占用的空间是最少的，但是耗时非常大
- np.tofile() 方法要处理数据的类型和形状，很烦，可以放弃了
- pickle() 的 .pkl 文件非常非常占空间，而且巨慢，直接放弃！！

## 非对齐的 ndarray 数据存储

In [2]:
import h5py

list_a = [[1,2,3], [4,5], [77]]
arr_a = np.asarray(list_a)
arr_a

array([[1, 2, 3], [4, 5], [77]], dtype=object)

In [ ]:
# 无法保存长度不同的 ndarray
f = h5py.File('data/arr_a.h5', 'w')
f['arr_a'] = arr_a  #　报错，每个元素都是 Object。

In [6]:
np.save('data/arr_a.npy', arr_a)
b = np.load('data/arr_a.npy')
print(b)

[[1, 2, 3] [4, 5] [77]]


通过上面的比较，我们可以看到 np.save() 非常强大呀，没办法，只好选择它了。

## 参考

- [python:numpy（文件存取）](http://www.cnblogs.com/dmir/p/5009075.html) 
- [HDF5在python上的使用](http://blog.csdn.net/leibaojiangjun1/article/details/53635353)